In [1]:
import numpy as np
import pandas as pd

In [12]:
import os
for dirname, _, filenames in os.walk('./kaggle/') :
    for filename in filenames :
        print(os.path.join(dirname, filename))

./kaggle/y_train.csv
./kaggle/X_train.csv
./kaggle/y_test.csv
./kaggle/X_test.csv


In [7]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor

In [14]:
# file read
xtrain = pd.read_csv('./kaggle/X_train.csv')
xtest = pd.read_csv('./kaggle/X_test.csv')
ytrain = pd.read_csv('./kaggle/y_train.csv')
ytest = pd.read_csv('./kaggle/y_test.csv')

print(xtrain.shape)
print(xtest.shape)
print(ytrain.shape)
print(ytest.shape)

(4960, 10)
(2672, 10)
(4960, 2)
(2672, 2)


In [16]:
# see sample
xtrain.head(5)

,carID,brand,model,year,transmission,mileage,fuelType,tax,mpg,engineSize
0,13207,hyundi,Santa Fe,2019,Semi-Auto,4223,Diesel,145.0,39.8,2.2
1,17314,vauxhall,GTC,2015,Manual,47870,Diesel,125.0,60.1,2.0
2,12342,audi,RS4,2019,Automatic,5151,Petrol,145.0,29.1,2.9
3,13426,vw,Scirocco,2016,Automatic,20423,Diesel,30.0,57.6,2.0
4,16004,skoda,Scala,2020,Semi-Auto,3569,Petrol,145.0,47.1,1.0


In [25]:
# train 데이터셋 하나로
df = pd.concat([xtrain, ytrain], axis = 1) # 1 왼쪽 오른쪽, 0 위 아래
df.drop(['carID'], axis = 1, inplace = True) # car id 컬럼 제거
df

,brand,model,year,transmission,mileage,fuelType,tax,mpg,engineSize,price
0,hyundi,Santa Fe,2019,Semi-Auto,4223,Diesel,145.0,39.8,2.2,31995
1,vauxhall,GTC,2015,Manual,47870,Diesel,125.0,60.1,2.0,7700
2,audi,RS4,2019,Automatic,5151,Petrol,145.0,29.1,2.9,58990
3,vw,Scirocco,2016,Automatic,20423,Diesel,30.0,57.6,2.0,12999
4,skoda,Scala,2020,Semi-Auto,3569,Petrol,145.0,47.1,1.0,16990
...,...,...,...,...,...,...,...,...,...,...
4955,merc,GL Class,2015,Automatic,24314,Diesel,125.0,56.6,2.1,17999
4956,bmw,6 Series,2017,Automatic,18000,Diesel,145.0,51.4,3.0,28900
4957,vw,CC,2015,Manual,84932,Diesel,30.0,60.1,2.0,8998
4958,audi,A7,2017,Semi-Auto,30150,Diesel,145.0,62.8,3.0,23198


In [26]:
# test dataset 합치기
xtest = pd.concat([xtest, ytest], axis = 1)
xtest.drop(['carID'], axis = 1, inplace = True)
xtest.head(5)

,brand,model,year,transmission,mileage,fuelType,tax,mpg,engineSize,price
0,merc,GLS Class,2017,Automatic,12046,Diesel,150.0,37.2,3.0,38000
1,vw,Amarok,2017,Automatic,37683,Diesel,260.0,36.2,3.0,23495
2,merc,GLS Class,2019,Automatic,10000,Diesel,145.0,34.0,3.0,59999
3,skoda,Scala,2019,Manual,3257,Petrol,145.0,49.6,1.0,16713
4,audi,RS6,2015,Semi-Auto,20982,Petrol,325.0,29.4,4.0,46000


In [28]:
# train, test 데이터셋 하나로 합쳐서 인덱스 재설정
df = pd.concat([df, xtest], axis = 0)
df.reset_index(inplace=True)

In [34]:
# 데이터셋 확인하는 함수
def describe(df) :
    columns = df.columns.to_list()
    ncol = df.describe().columns.to_list()
    ccol = []
    for i in columns :
        if(ncol.count(i) == 0) :
            ccol.append(i)
        else :
            continue
    print('DataFrame의 모든 컬럼 : ')
    print(columns)
    print('')
    print('전체 컬럼 갯수 : ')
    print(len(columns))
    print('')
    print('numerical 컬럼 목록 : ')
    print(ncol)
    print('')
    print('numerical 컬럼 갯수 : ')
    print(len(ncol))
    print('')
    print('범주형 컬럼 목록 : ')
    print(ccol)
    print('')
    print('범주형 컬럼 갯수 : ')
    print(len(ccol))
    print('')
    print('===================================================')
    print('')
    print('Null 값')
    print(df.isnull().sum())
    print('')
    print('Unique value ')
    print(df.nunique())
    print('')
    print('Basic Statistics and Measures for Numerical Columns:')
    print(df.describe().T)
    print('')
    print('Other Relevant Metadata Regarding the Dataframe:')
    print(df.info())

In [35]:
describe(df)

DataFrame의 모든 컬럼 : 
['index', 'brand', 'model', 'year', 'transmission', 'mileage', 'fuelType', 'tax', 'mpg', 'engineSize', 'price']

전체 컬럼 갯수 : 
11

numerical 컬럼 목록 : 
['index', 'year', 'mileage', 'tax', 'mpg', 'engineSize', 'price']

numerical 컬럼 갯수 : 
7

범주형 컬럼 목록 : 
['brand', 'model', 'transmission', 'fuelType']

범주형 컬럼 갯수 : 
4


Null 값
index           0
brand           0
model           0
year            0
transmission    0
mileage         0
fuelType        0
tax             0
mpg             0
engineSize      0
price           0
dtype: int64

Unique value 
index           4960
brand              9
model             90
year              25
transmission       4
mileage         5790
fuelType           5
tax               43
mpg              128
engineSize        38
price           3032
dtype: int64

Basic Statistics and Measures for Numerical Columns:
             count          mean           std     min       25%      50%  \
index       7632.0   2078.980084   1355.983560     0.0   